In [7]:
import logging
import telebot
from telebot import types
from telegram_bot_pagination import InlineKeyboardPaginator

import operations
from config import DB_ADDRESS_OPERATIONS, TOKEN

logging.basicConfig(filename='operations.log', format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO)



bot=telebot.TeleBot(TOKEN)

welcome_message = """
Привет! Это первая попытка написать телеграм-бот для отслеживания изменения цен в Перекрестке. \
Принцип работы следующий:\n\
- вы находите на сайте www.perekrestok.ru интересующий вас товар\n\
- нажимаете кнопку "Добавить продукт" и копируете ссылку на сайт\n\
- отправляете ссылку боту.\n\
Теперь цена на этот товар проверяется дважды в день и записывается в базу данных. В любой момент вы \
можете посмотреть список отслеживаемых товаров по кнопке "Мой список".\n
Когда вы идете в магазин, нажмите кнопку "Что по акции?". Бот пришлет вам список тех товаров из вашего списка, \
которые в данный момент продаются по акции.

По команде /demo вы можете подключить небольшую демо-базу на 6 продуктов.\n

Это первый рабочий прототип. Если что-то работает не так, или у вас есть идеи, я буду признателен за /feedback
"""

def add_item(message):
    url = message.text.strip()
    user = message.from_user.username
    
    result = operations.add_item_to_database(url, user)
    if result == 'Already_exist':
        bot.send_message(message.chat.id, 'Этот продукт уже находится в вашем списке наблюдения',
                        reply_markup=kboard)
    elif result == 'Added':
        bot.send_message(message.chat.id, 'Продукт добавлен в ваш список наблюдения',
                        reply_markup=kboard)
    else:
        bot.send_message(message.chat.id, 'Ошибка добавления - ссылка не может быть обработана',
                        reply_markup=kboard)


def remove_item(message):
    ID = message.text.strip()
    user = message.from_user.username
    result = operations.remove_item_from_watch(ID, user)
    if result == "Not_in_list":
        bot.send_message(message.chat.id, f"ID {ID} не найден в вашем списке наблюдения",
                        reply_markup=kboard)
    elif result == "Removed":
        bot.send_message(message.chat.id, f"ID {ID} удален из списка наблюдения",
                        reply_markup=kboard)
    else:
        bot.send_message(message.chat.id, f"ID {ID} ворует-убивает гусей",
                        reply_markup=kboard)

        
def get_items(message, isCallback=False, page=1):
    user = message.from_user.username
    user = 'root'
    result = operations.get_list_of_items_on_watch(user)
    if not result:
        answer = ["У вас нет продуктов в списке наблюдения"]
    else:    
        answer = []
        for i in range(0, len(result), 12):
            batch = result[i:i+12]
            answer.append("\n".join([f"id {i['id']}: {i['name']}" for i in batch]))
    
    paginator = InlineKeyboardPaginator(
        len(answer),
        current_page=page,
        data_pattern='items#{page}'
    )
    if isCallback:
        bot.edit_message_text(
            chat_id=message.chat.id,
            text=answer[page-1],
            message_id=message.message_id,
            reply_markup=paginator.markup,
            parse_mode='Markdown'
        )
    else:
        bot.send_message(
            message.chat.id,
            answer[page-1],
            reply_markup=paginator.markup,
            parse_mode='Markdown'
        )

        
def get_profits(message, isCallback=False, page=1):
    user = message.from_user.username
    user = 'root'
    result = operations.get_list_of_profits_for_user(user, abs_profit_threshold=10)
    if not result:
        answer = ["Сегодня не ваш день - по скидонькам ничего нет :("]
    else:
        answer = []
        for i in range(0, len(result), 5):
            batch = result[i:i+5]
            formatted_message = "\n".join(
                [f"*{i['name']}*\n\
    Сейчас: {i['price_last']}р, в среднем: {i['price_mean']}р\n\
    Профит: {i['abs_profit']}р ({i['relt_profit']})\n" for i in batch]
            )
            answer.append(formatted_message)
        
    paginator = InlineKeyboardPaginator(
        len(answer),
        current_page=page,
        data_pattern='profits#{page}'
    )
    if isCallback:
        bot.edit_message_text(
            chat_id=message.chat.id,
            text=answer[page-1],
            message_id=message.message_id,
            reply_markup=paginator.markup,
            parse_mode='Markdown'
        )
    else:
        bot.send_message(
            message.chat.id,
            answer[page-1],
            reply_markup=paginator.markup,
            parse_mode='Markdown'
        )

        
def feedback(message):
    with open('feedback.txt', 'a') as file:
        file.write(f"Message from user {message.from_user.username}:\n")
        file.write(message.text)
        file.write('\n')
        bot.send_message(message.chat.id, "Cпасибо! Мы обратим на это внимание",
                        reply_markup=kboard)

        
def add_demo(message):
    user = message.from_user.username
    if operations.add_demo_for_user(user):
        bot.send_message(message.chat.id, "Демо-база подключена", reply_markup=kboard)
    else:
        bot.send_message(message.chat.id, "Ошибка подключения базы", reply_markup=kboard)

        
        
# set standard keyboard
kboard=types.ReplyKeyboardMarkup(resize_keyboard=True)
button_add=types.KeyboardButton("Добавить продукт")
button_show=types.KeyboardButton("Мой список")
button_remove=types.KeyboardButton("Удалить продукт")
button_get=types.KeyboardButton("А что по акции?")

kboard.row(button_add, button_show, button_remove)
kboard.row(button_get)

# set blank keyboard
kboard_blank=telebot.types.ReplyKeyboardRemove()



@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, welcome_message, reply_markup=kboard)
    
@bot.message_handler(commands=['feedback'])
def feedback_message(message):
    bot.send_message(message.chat.id,"Спасибо за фидбек! Опишите, пожалуйста, проблему (текстом)",
                    reply_markup=kboard_blank)
    bot.register_next_step_handler(message, feedback)

@bot.message_handler(commands=['demo'])
def demo_message(message):
    add_demo(message)
        
@bot.message_handler(content_types='text')
def message_reply(message):
    if message.text=="Добавить продукт":
        bot.send_message(message.chat.id, "Пришлите URL продукта, который хотите добавить",
                         reply_markup=kboard_blank)
        bot.register_next_step_handler(message, add_item)
        
    elif message.text=="Мой список":
        get_items(message)

    elif message.text=="Удалить продукт":
        bot.send_message(message.chat.id, "ID продукта, который вы хотите удалить?",
                         reply_markup=kboard_blank)
        bot.register_next_step_handler(message, remove_item)
        
    elif message.text=="А что по акции?":
        get_profits(message)
        
    else:
        ...
        
@bot.callback_query_handler(func=lambda call: call.data.split('#')[0]=='items')
def items_page_callback(call):
    page = int(call.data.split('#')[1])
    get_items(call.message, True, page)
    
@bot.callback_query_handler(func=lambda call: call.data.split('#')[0]=='profits')
def profits_page_callback(call):
    page = int(call.data.split('#')[1])
    get_profits(call.message, True, page)

    
bot.infinity_polling()


2022-11-06 22:02:16,767 (__init__.py:580 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-11-06 22:02:16,769 (__init__.py:582 MainThread) ERROR - TeleBot: "Break infinity polling"
